#### This notebook is to understand the Apriori Algorithm

In [1]:
from collections import Counter,OrderedDict

In [2]:
# This Function is used to generate all the subset of any set
def genSubsets(l):
    powerSetSize = 2 ** len(l)
    powerSet = []
    for i in range(1,powerSetSize):
        tempEle = []
        for j in range(len(l)):
            binFlagInd = i & (1 << j)
            if binFlagInd:
                tempEle.append(l[j])
        powerSet.append(tempEle)
    return powerSet

In [3]:
# Apriori Algorithm Implementation
def genCandidate(Fk1): #Fk1 indicates F(k-1), it is a list of lists
    Ck = []
    k1  = len(Fk1[0])

    # COMBINE STEP
    for i in range(len(Fk1)-1):
        for j in range(i+1,len(Fk1)):
            f1,f2 = Fk1[i],Fk1[j]

            if f1[:len(f1)-1] == f2[:len(f2)-1] and f1[-1] < f2[-1]:
                tempC = f1 + [f2[-1]]

                # PRUNING STEP
                subset = genSubsets(tempC)
                appendSts = True
                for item in subset:
                    if len(item) == k1 and item not in Fk1:
                        appendSts = False
                if appendSts:
                    Ck.append(tempC)              
    return Ck

In [4]:

    
def initPass(txList): # list of transactions, most possibly a dict
    allTx = [item for tx in txList for item in tx]
    allTx.sort()
    cntr =  OrderedDict()
    for tx in allTx:
        cntr[tx] = cntr.get(tx,0) + 1

    return cntr



In [5]:
def searchInT(t,candid):
    found = True
    for eachCandid in candid:
        if eachCandid not in t:
            found = False

    return found


In [6]:
def apriori(T,minSup):
    finalSet = []
    c1 = initPass(T)
    f = [[item] for item in c1.keys() if c1[item]/len(T) >= minSup] # f1
    
    for item in f:
        finalSet.append(item)

    while len(f) != 0:
        Ck = genCandidate(f)
        # print("Ck")
        # print(Ck)
        freqDict = {}
        for t in T:
            for candidate in Ck:
                if searchInT(t,candidate):
                    freqDict[tuple(candidate)] = freqDict.get(tuple(candidate),0) + 1
        # print("freqDict")
        # print(freqDict)
        f = []
        for c in freqDict.keys():
            if freqDict[c]/len(T)>= minSup:
                f.append(list(c))

        # print("f")
        # print(f)
        if len(f) != 0:
            f = sorted(f,key=lambda x : (len(x),*x))
            for item in f:
                finalSet.append(item)
    # print(finalSet)
    return finalSet

In [7]:
T = [
    ['1', '3', '4'],
    ['2', '3', '5'],
    ['1', '2', '3', '5'],
    ['1','2','5']
    ] 

print(apriori(T,0.5))

[['1'], ['2'], ['3'], ['5'], ['1', '2'], ['1', '3'], ['1', '5'], ['2', '3'], ['2', '5'], ['3', '5'], ['1', '2', '5'], ['2', '3', '5']]


In [8]:
T2=[['Bread', 'Milk'], 
            ['Bread', 'Diapers', 'Beer', 'Eggs'], 
            ['Milk', 'Diapers', 'Beer', 'Coke'], 
            ['Bread', 'Milk', 'Diapers', 'Beer'], 
            ['Bread', 'Milk', 'Diapers', 'Coke']]
F=apriori(T2,0.5)

In [9]:
print(F)

[['Beer'], ['Bread'], ['Diapers'], ['Milk'], ['Beer', 'Diapers'], ['Bread', 'Diapers'], ['Bread', 'Milk'], ['Diapers', 'Milk']]


In [10]:
T3 = [
    ['beer', 'wine', 'cheese'],
    ['beer', 'potato chips'],
    ['eggs', 'flower', 'butter', 'cheese'],
    ['eggs', 'flower', 'butter', 'beer', 'potato chips'],
    ['wine', 'cheese'],
    ['potato chips'],
    ['eggs', 'flower', 'butter', 'wine', 'cheese'],
    ['eggs', 'flower', 'butter', 'beer', 'potato chips'],
    ['wine', 'beer'],
    ['beer', 'potato chips'],
    ['butter', 'eggs'],
    ['beer', 'potato chips'],
    ['flower', 'eggs'],
    ['beer', 'potato chips'],
    ['eggs', 'flower', 'butter', 'wine', 'cheese'],
    ['beer', 'wine', 'potato chips', 'cheese'],
    ['wine', 'cheese'],
    ['beer', 'potato chips'],
    ['wine', 'cheese'],
    ['beer', 'potato chips']
]

In [15]:
F=apriori(T3,0.3)
print(F)

[['beer'], ['butter'], ['cheese'], ['eggs'], ['flower'], ['potato chips'], ['wine'], ['beer', 'potato chips'], ['butter', 'eggs'], ['cheese', 'wine'], ['eggs', 'flower']]


In [16]:
#!pip install pyECLAT

  Found existing installation: tqdm 4.32.1
    Uninstalling tqdm-4.32.1:
      Successfully uninstalled tqdm-4.32.1
  Found existing installation: numpy 1.17.1
    Uninstalling numpy-1.17.1:
      Successfully uninstalled numpy-1.17.1


In [18]:
import pandas as pd

# you simply convert the transaction list into a dataframe
data = pd.DataFrame(T3)
data

,0,1,2,3,4
0,beer,wine,cheese,None,None
1,beer,potato chips,None,None,None
2,eggs,flower,butter,cheese,None
3,eggs,flower,butter,beer,potato chips
4,wine,cheese,None,None,None
5,potato chips,None,None,None,None
6,eggs,flower,butter,wine,cheese
7,eggs,flower,butter,beer,potato chips
8,wine,beer,None,None,None
9,beer,potato chips,None,None,None


In [20]:
# we are looking for itemSETS
# we do not want to have any individual products returned
min_n_products = 2

# we want to set min support to 7
# but we have to express it as a percentage
min_support = 7/len(T3)

# we have no limit on the size of association rules
# so we set it to the longest transaction
max_length = max([len(x) for x in T3])

In [21]:
from pyECLAT import ECLAT

# create an instance of eclat
my_eclat = ECLAT(data=data, verbose=True)

# fit the algorithm
rule_indices, rule_supports = my_eclat.fit(min_support=min_support,
                                           min_combination=min_n_products,
                                           max_combination=max_length)

100%|██████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 1167.64it/s]


Combination 2 by 2


10it [00:00, 203.80it/s]


Combination 3 by 3


10it [00:00, 209.08it/s]


Combination 4 by 4


5it [00:00, 135.26it/s]


Combination 5 by 5


1it [00:00, 129.29it/s]


In [22]:
print(rule_supports)

{'cheese & wine': 0.35, 'potato chips & beer': 0.45}
